In [3]:

import sys
import os
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, train_and_evaluate_logistic_regression, train_and_evaluate_linear_svm, train_and_evaluate_non_linear_svm, train_and_evaluate_decision_tree, train_and_evaluate_random_forest, train_and_evaluate_xgboost
from configs.config import DATASET_PATH, FEATURES_DIR, ITW_DATASET_PATH, MODELS_PATH

from utils.utils import grid_search_joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np


import matplotlib.pyplot as plt
import joblib


### Parquet paths

In [4]:
train_data_path = os.path.join(FEATURES_DIR, "training_features_mean_20_128_256_128.parquet")
val_data_path = os.path.join(FEATURES_DIR, "testing_features_mean_20_128_256_128.parquet")
test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features', "itw_features_mean_20_128_256_128_trimmed_loudness_normalized.parquet")

#no mel features
train_data_path_no_mel = os.path.join(FEATURES_DIR, "training_features_mean_20_128_256_128_no_mel.parquet")
test_data_path_no_mel = os.path.join(FEATURES_DIR, "testing_features_mean_20_128_256_128_no_mel.parquet")
itw_data_path_no_mel = os.path.join(ITW_DATASET_PATH, 'normalized_features', "itw_features_mean_20_128_256_128_no_mel_trimmed_loudness_normalized.parquet")

### Logistic Regression

In [ ]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        class_weight={0:1, 1:5},
        random_state=42,
        n_jobs=1,
    )),
])

param_grid = {
    "clf__solver": ["saga"],
    "clf__penalty": ["l1","l2"],
    "clf__C": np.logspace(-3, 2, 10),
    "clf__max_iter": [1000],
}

(
    final_model, 
    test_metrics, 
    val_metrics, 
    best_params, 
    val_results, 
    metadata, 
    feature_names  
) = grid_search_joblib(
    model,
    param_grid,
    train_data_path,
    val_data_path,
    test_data_path,
    n_jobs=20
)

print(f'Best parameters:{best_params}')
print(f'Resluts on validation data:{val_metrics}')
print(f'Resluts on test data:{test_metrics}')

save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "logistic_reg",),
    model_params=best_params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)


Number of fits: 20 with n_jobs=20 parallel jobs


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:   41.4s remaining:  6.2min
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:  1.6min remaining:  4.9min
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed:  1.6min remaining:  1.3min
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:  2.0min remaining:   50.9s
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed:  2.0min remaining:   21.4s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:  2.1min finished


[1] {'clf__C': np.float64(0.001), 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'clf__solver': 'saga'} | val_acc=0.5645 | val_f1=0.4522
[2] {'clf__C': np.float64(0.001), 'clf__max_iter': 1000, 'clf__penalty': 'l2', 'clf__solver': 'saga'} | val_acc=0.6349 | val_f1=0.5753
[3] {'clf__C': np.float64(0.003593813663804626), 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'clf__solver': 'saga'} | val_acc=0.6398 | val_f1=0.5844
[4] {'clf__C': np.float64(0.003593813663804626), 'clf__max_iter': 1000, 'clf__penalty': 'l2', 'clf__solver': 'saga'} | val_acc=0.6800 | val_f1=0.6432
[5] {'clf__C': np.float64(0.01291549665014884), 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'clf__solver': 'saga'} | val_acc=0.6899 | val_f1=0.6589
[6] {'clf__C': np.float64(0.01291549665014884), 'clf__max_iter': 1000, 'clf__penalty': 'l2', 'clf__solver': 'saga'} | val_acc=0.7095 | val_f1=0.6848
[7] {'clf__C': np.float64(0.046415888336127795), 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'clf__solver': 'saga'} | val_acc=0.716

c:\Users\geon9\MSc\audio-deepfake-detection\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


'c:\\Users\\geon9\\MSc\\audio-deepfake-detection\\notebooks\\experiments\\final\\logistic_reg\\exp_20260207_192945'

### Linear SVM

In [8]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm",  LinearSVC(max_iter=20000, class_weight={0:1, 1:5}, random_state=42)),
])

param_grid = {
    "svm__C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
}

(
    final_model, 
    test_metrics, 
    val_metrics, 
    best_params, 
    val_results, 
    metadata, 
    feature_names 
) = grid_search_joblib(
    model,
    param_grid,
    train_data_path,
    val_data_path,
    test_data_path,
    n_jobs=20
)

print(f'Best parameters:{best_params}')
print(f'Resluts on validation data:{val_metrics}')
print(f'Resluts on test data:{test_metrics}')

save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "linear_svm"),
    model_params=best_params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)

Number of fits: 7 with n_jobs=20 parallel jobs


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=20)]: Done   2 out of   7 | elapsed:    4.8s remaining:   12.2s
[Parallel(n_jobs=20)]: Done   3 out of   7 | elapsed:    5.8s remaining:    7.8s
[Parallel(n_jobs=20)]: Done   4 out of   7 | elapsed:    7.2s remaining:    5.4s
[Parallel(n_jobs=20)]: Done   5 out of   7 | elapsed:    7.3s remaining:    2.9s
[Parallel(n_jobs=20)]: Done   7 out of   7 | elapsed:    7.4s finished


[1] {'svm__C': 0.001} | val_acc=0.7013 | val_f1=0.6730
[2] {'svm__C': 0.01} | val_acc=0.7052 | val_f1=0.6815
[3] {'svm__C': 0.1} | val_acc=0.7044 | val_f1=0.6810
[4] {'svm__C': 1} | val_acc=0.7037 | val_f1=0.6803
[5] {'svm__C': 10} | val_acc=0.7041 | val_f1=0.6809
[6] {'svm__C': 100} | val_acc=0.7041 | val_f1=0.6809
[7] {'svm__C': 1000} | val_acc=0.7041 | val_f1=0.6809

Best validation result:
  params: {'svm__C': 0.01}
  f1_macro: 0.6815
Best parameters:{'svm__C': 0.01}
Resluts on validation data:{'accuracy': 0.7052222701769529, 'precision': 0.7741288498521771, 'recall': 0.6993488243801345, 'f1': 0.6815274100862034, 'roc_auc': 0.890393016355802}
Resluts on test data:{'accuracy': 0.7200406014083613, 'precision': 0.7074885236262592, 'recall': 0.718060558012356, 'f1': 0.70969285375347, 'roc_auc': 0.7810866983889688}
Experiment saved to: c:\Users\geon9\MSc\audio-deepfake-detection\notebooks\experiments\final\linear_svm\exp_20260207_193304


'c:\\Users\\geon9\\MSc\\audio-deepfake-detection\\notebooks\\experiments\\final\\linear_svm\\exp_20260207_193304'

### RBF SVM

In [13]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(
        class_weight={0: 1, 1: 5},
        cache_size=2000,
        random_state=42
    )),
])

param_grid_rbf = {
    "svm__kernel": ["rbf"],
    "svm__C": [0.1, 1, 10, 100],
    "svm__gamma": ["scale", 0.01, 0.1],
}

(
    final_model_rbf, 
    test_metrics_rbf, 
    val_metrics_rbf, 
    best_params_rbf, 
    val_results_rbf, 
    metadata_rbf, 
    feature_names_rbf 
) = grid_search_joblib(
    model,
    param_grid_rbf,
    train_data_path,
    val_data_path,
    test_data_path,
    n_jobs=20
)
print(f'Best parameters:{best_params_rbf}')
print(f'Resluts on validation data:{val_metrics_rbf}')
print(f'Resluts on test data:{test_metrics_rbf}')

save_experiment(
    model=final_model_rbf,
    metrics=test_metrics_rbf,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "rbf_svm"),
    model_params=best_params_rbf,
    feature_names=feature_names_rbf,
    metadata_extra=metadata_rbf,
    val_results=val_results_rbf,
)

Number of fits: 12 with n_jobs=20 parallel jobs


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  3.8min
[Parallel(n_jobs=20)]: Done   3 out of  12 | elapsed:  3.9min remaining: 11.6min
[Parallel(n_jobs=20)]: Done   5 out of  12 | elapsed:  5.4min remaining:  7.5min
[Parallel(n_jobs=20)]: Done   7 out of  12 | elapsed:  7.2min remaining:  5.1min
[Parallel(n_jobs=20)]: Done   9 out of  12 | elapsed: 22.1min remaining:  7.4min
[Parallel(n_jobs=20)]: Done  12 out of  12 | elapsed: 28.6min finished


[1] {'svm__C': 0.1, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'} | val_acc=0.7555 | val_f1=0.7545
[2] {'svm__C': 0.1, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'} | val_acc=0.7611 | val_f1=0.7607
[3] {'svm__C': 0.1, 'svm__gamma': 0.1, 'svm__kernel': 'rbf'} | val_acc=0.5142 | val_f1=0.3455
[4] {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'} | val_acc=0.6655 | val_f1=0.6427
[5] {'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'} | val_acc=0.6774 | val_f1=0.6586
[6] {'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'rbf'} | val_acc=0.6155 | val_f1=0.5431
[7] {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'} | val_acc=0.6075 | val_f1=0.5549
[8] {'svm__C': 10, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'} | val_acc=0.6465 | val_f1=0.6171
[9] {'svm__C': 10, 'svm__gamma': 0.1, 'svm__kernel': 'rbf'} | val_acc=0.6265 | val_f1=0.5612
[10] {'svm__C': 100, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'} | val_acc=0.6090 | val_f1=0.5571
[11] {'svm__C': 100, 'svm__gamma': 0.01, 'svm__ke

'c:\\Users\\geon9\\MSc\\audio-deepfake-detection\\notebooks\\experiments\\final\\rbf_svm\\exp_20260207_200752'

### Poly SVM

In [4]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(
        class_weight={0: 1, 1: 5},
        cache_size=2000,
        random_state=42
    )),
])

param_grid_poly = {
    "svm__kernel": ["poly"],
    "svm__degree": [2, 3],
    "svm__C": [0.1, 1, 10],
    "svm__gamma": ["scale", 0.01],
    "svm__coef0": [0.0, 1.0],
}

(
    final_model_poly, 
    test_metrics_poly, 
    val_metrics_poly, 
    best_params_poly, 
    val_results_poly, 
    metadata_poly, 
    feature_names_poly 
) = grid_search_joblib(
    model,
    param_grid_poly,
    train_data_path,
    val_data_path,
    test_data_path,
    n_jobs=20
)
print(f'Best parameters:{best_params_poly}')
print(f'Resluts on validation data:{val_metrics_poly}')
print(f'Resluts on test data:{test_metrics_poly}')

save_experiment(
    model=final_model_poly,
    metrics=test_metrics_poly,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "final", "poly_svm"),
    model_params=best_params_poly,
    feature_names=feature_names_poly,
    metadata_extra=metadata_poly,
    val_results=val_results_poly,
)

Number of fits: 24 with n_jobs=20 parallel jobs


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  2.0min
[Parallel(n_jobs=20)]: Done   3 out of  24 | elapsed:  2.3min remaining: 16.1min
[Parallel(n_jobs=20)]: Done   6 out of  24 | elapsed:  2.8min remaining:  8.5min
[Parallel(n_jobs=20)]: Done   9 out of  24 | elapsed:  3.1min remaining:  5.2min
[Parallel(n_jobs=20)]: Done  12 out of  24 | elapsed:  3.2min remaining:  3.2min
[Parallel(n_jobs=20)]: Done  15 out of  24 | elapsed:  3.3min remaining:  2.0min
[Parallel(n_jobs=20)]: Done  18 out of  24 | elapsed:  3.5min remaining:  1.2min
[Parallel(n_jobs=20)]: Done  21 out of  24 | elapsed:  4.4min remaining:   37.5s
[Parallel(n_jobs=20)]: Done  24 out of  24 | elapsed:  5.9min finished


[1] {'svm__C': 0.1, 'svm__coef0': 0.0, 'svm__degree': 2, 'svm__gamma': 'scale', 'svm__kernel': 'poly'} | val_acc=0.6096 | val_f1=0.6057
[2] {'svm__C': 0.1, 'svm__coef0': 0.0, 'svm__degree': 2, 'svm__gamma': 0.01, 'svm__kernel': 'poly'} | val_acc=0.7024 | val_f1=0.7015
[3] {'svm__C': 0.1, 'svm__coef0': 0.0, 'svm__degree': 3, 'svm__gamma': 'scale', 'svm__kernel': 'poly'} | val_acc=0.4825 | val_f1=0.3867
[4] {'svm__C': 0.1, 'svm__coef0': 0.0, 'svm__degree': 3, 'svm__gamma': 0.01, 'svm__kernel': 'poly'} | val_acc=0.5811 | val_f1=0.5475
[5] {'svm__C': 0.1, 'svm__coef0': 1.0, 'svm__degree': 2, 'svm__gamma': 'scale', 'svm__kernel': 'poly'} | val_acc=0.7400 | val_f1=0.7400
[6] {'svm__C': 0.1, 'svm__coef0': 1.0, 'svm__degree': 2, 'svm__gamma': 0.01, 'svm__kernel': 'poly'} | val_acc=0.7054 | val_f1=0.6974
[7] {'svm__C': 0.1, 'svm__coef0': 1.0, 'svm__degree': 3, 'svm__gamma': 'scale', 'svm__kernel': 'poly'} | val_acc=0.6791 | val_f1=0.6674
[8] {'svm__C': 0.1, 'svm__coef0': 1.0, 'svm__degree': 3, 

'c:\\Users\\geon9\\MSc\\audio-deepfake-detection\\notebooks\\experiments\\final\\poly_svm\\exp_20260207_201742'

### Sigmoid Kernel

In [5]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(
        class_weight={0: 1, 1: 5},
        cache_size=2000,
        random_state=42
    )),
])

param_grid_sigmoid = {
    "svm__kernel": ["sigmoid"],
    "svm__C": [0.01, 0.1, 1],
    "svm__gamma": ["scale", 0.01],
    "svm__coef0": [-1.0, 0.0, 1.0],
}

(
    final_model_sigmoid, 
    test_metrics_sigmoid, 
    val_metrics_sigmoid, 
    best_params_sigmoid, 
    val_results_sigmoid, 
    metadata_sigmoid, 
    feature_names_sigmoid 
) = grid_search_joblib(
    model,
    param_grid_sigmoid,
    train_data_path,
    val_data_path,
    test_data_path,
    n_jobs=20
)

print(f'Best parameters:{best_params_sigmoid}')
print(f'Resluts on validation data:{val_metrics_sigmoid}')
print(f'Resluts on test data:{test_metrics_sigmoid}')

save_experiment(
    model=final_model_sigmoid,
    metrics=test_metrics_sigmoid,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "sigmoid_svm"),
    model_params=best_params_sigmoid,
    feature_names=feature_names_sigmoid,
    metadata_extra=metadata_sigmoid,
    val_results=val_results_sigmoid,
)

Number of fits: 18 with n_jobs=20 parallel jobs


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   3 out of  18 | elapsed: 13.5min remaining: 67.8min
[Parallel(n_jobs=20)]: Done   5 out of  18 | elapsed: 14.6min remaining: 38.1min
[Parallel(n_jobs=20)]: Done   7 out of  18 | elapsed: 14.8min remaining: 23.3min
[Parallel(n_jobs=20)]: Done   9 out of  18 | elapsed: 16.0min remaining: 16.0min
[Parallel(n_jobs=20)]: Done  11 out of  18 | elapsed: 16.7min remaining: 10.6min
[Parallel(n_jobs=20)]: Done  13 out of  18 | elapsed: 18.0min remaining:  6.9min
[Parallel(n_jobs=20)]: Done  15 out of  18 | elapsed: 18.4min remaining:  3.7min
[Parallel(n_jobs=20)]: Done  18 out of  18 | elapsed: 18.7min finished


[1] {'svm__C': 0.01, 'svm__coef0': -1.0, 'svm__gamma': 'scale', 'svm__kernel': 'sigmoid'} | val_acc=0.5334 | val_f1=0.3882
[2] {'svm__C': 0.01, 'svm__coef0': -1.0, 'svm__gamma': 0.01, 'svm__kernel': 'sigmoid'} | val_acc=0.5766 | val_f1=0.5033
[3] {'svm__C': 0.01, 'svm__coef0': 0.0, 'svm__gamma': 'scale', 'svm__kernel': 'sigmoid'} | val_acc=0.5451 | val_f1=0.4179
[4] {'svm__C': 0.01, 'svm__coef0': 0.0, 'svm__gamma': 0.01, 'svm__kernel': 'sigmoid'} | val_acc=0.5537 | val_f1=0.4454
[5] {'svm__C': 0.01, 'svm__coef0': 1.0, 'svm__gamma': 'scale', 'svm__kernel': 'sigmoid'} | val_acc=0.5227 | val_f1=0.3652
[6] {'svm__C': 0.01, 'svm__coef0': 1.0, 'svm__gamma': 0.01, 'svm__kernel': 'sigmoid'} | val_acc=0.5259 | val_f1=0.3722
[7] {'svm__C': 0.1, 'svm__coef0': -1.0, 'svm__gamma': 'scale', 'svm__kernel': 'sigmoid'} | val_acc=0.7268 | val_f1=0.7137
[8] {'svm__C': 0.1, 'svm__coef0': -1.0, 'svm__gamma': 0.01, 'svm__kernel': 'sigmoid'} | val_acc=0.5719 | val_f1=0.5306
[9] {'svm__C': 0.1, 'svm__coef0': 

'c:\\Users\\geon9\\MSc\\audio-deepfake-detection\\notebooks\\experiments\\final\\sigmoid_svm\\exp_20260207_204305'

### Decission Tree

In [6]:
model = DecisionTreeClassifier(random_state=42)

params = {
    "max_depth": [x for x in range(5, 20)],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "class_weight": [{0: 1, 1: 5}, None],
    "criterion": ["gini", "entropy"],
    "ccp_alpha": [0.0, 1e-4, 1e-3, 1e-2],
}


(
    final_model,
    test_metrics,
    val_metrics,
    best_params,
    val_results,
    metadata,
    feature_names
) = grid_search_joblib(
    model=model,
    param_grid=params,
    train_path=train_data_path,
    val_path=val_data_path,
    test_path=test_data_path,
    scoring="f1_macro",
    verbose=1,
    n_jobs=-1
)

save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "Dtree",),
    model_params=params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)
print(metadata)
print(test_metrics)

Number of fits: 2160 with n_jobs=-1 parallel jobs


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:  

[1] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2} | val_acc=0.9109 | val_f1=0.9100
[2] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5} | val_acc=0.9109 | val_f1=0.9100
[3] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10} | val_acc=0.9109 | val_f1=0.9100
[4] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2} | val_acc=0.9109 | val_f1=0.9100
[5] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5} | val_acc=0.9109 | val_f1=0.9100
[6] {'ccp_alpha': 0.0, 'class_weight': {0: 1, 1: 5}, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10} | val_acc=0

### Random Forest

In [ ]:

from sklearn.impute import SimpleImputer


base_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('rf', RandomForestClassifier(random_state=42))
])

params = {
    # CORE ANTI-OVERFITTING: Very shallow trees
    "max_depth": [3, 4, 5],  # Much shallower than DTree's 10
    
    # LEARNING: Slow and steady
    "learning_rate": [0.01, 0.03, 0.05],
    "n_estimators": [300, 500, 700],  # More trees, lower LR
    
    # REGULARIZATION: Aggressive to prevent dataset artifacts
    "min_child_weight": [5, 10, 20],  # Higher than default
    "gamma": [0.1, 0.3, 0.5],  # Minimum loss reduction
    "lambda": [2, 5, 10],  # L2 regularization
    "alpha": [0, 0.5, 1],  # L1 regularization
    
    # SAMPLING: Reduce correlation and overfitting
    "subsample": [0.6, 0.7, 0.8],  # Row sampling
    "colsample_bytree": [0.6, 0.7, 0.8],  # Feature sampling
    
    # CLASS IMBALANCE: More moderate than 1:5
    "scale_pos_weight": [1, 2, 3],  # or calculate actual ratio
    
    # Fixed params
    "objective": ["binary:logistic"],
    "eval_metric": ["logloss"],
    "tree_method": ["hist"],  # Fast
    "random_state": [42],
}

(
    final_model,
    test_metrics,
    val_metrics,
    best_params,
    val_results,
    metadata,
    feature_names
) = grid_search_joblib(
    model=base_pipe,
    param_grid=param_grid,
    train_path=train_data_path,
    val_path=val_data_path,
    test_path=test_data_path,
    n_jobs=-1,
    verbose=2
)
    
save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "final", "RF",),
    model_params=best_params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)

print("Metadata:", metadata)
print("Metrics:", test_metrics)

### XGBoost

#### XGBoost Mel features


In [ ]:
base_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('xgb', XGBClassifier(eval_metric='aucpr', random_state=42))
])

params = {
    # CORE ANTI-OVERFITTING: Very shallow trees
    "max_depth": [3, 4, 5],  # Much shallower than DTree's 10
    
    # LEARNING: Slow and steady
    "learning_rate": [0.01, 0.03, 0.05],
    "n_estimators": [300, 500, 700],  # More trees, lower LR
    
    # REGULARIZATION: Aggressive to prevent dataset artifacts
    "min_child_weight": [5, 10, 20],  # Higher than default
    "gamma": [0.1, 0.3, 0.5],  # Minimum loss reduction
    "lambda": [2, 5, 10],  # L2 regularization
    "alpha": [0, 0.5, 1],  # L1 regularization
    
    # SAMPLING: Reduce correlation and overfitting
    "subsample": [0.6, 0.7, 0.8],  # Row sampling
    "colsample_bytree": [0.6, 0.7, 0.8],  # Feature sampling
    
    # CLASS IMBALANCE: More moderate than 1:5
    "scale_pos_weight": [1, 2, 3],  # or calculate actual ratio
    
    # Fixed params
    "objective": ["binary:logistic"],
    "eval_metric": ["logloss"],
    "tree_method": ["hist"],  # Fast
    "random_state": [42],
}

(
    final_model,
    test_metrics,
    val_metrics,
    best_params,
    val_results,
    metadata,
    feature_names
) = grid_search_joblib(
    model=base_pipe,
    param_grid=params,
    train_path=train_data_path,
    val_path=val_data_path,
    test_path=test_data_path,
    n_jobs=-1,
    verbose=2
)

save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "XGB"),
    model_params=best_params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)

print("Metadata:", metadata)
print("Metrics:", test_metrics)

#### XGBoost without mel features

In [ ]:
base_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('xgb', XGBClassifier(eval_metric='aucpr', random_state=42))
])

param_grid = {
    # CORE ANTI-OVERFITTING: Very shallow trees
    "xgb__max_depth": [3, 4, 5],  # ← xgb__ prefix
    
    # LEARNING: Slow and steady
    "xgb__learning_rate": [0.01, 0.03, 0.05],
    "xgb__n_estimators": [300, 500, 700],
    
    # REGULARIZATION: Aggressive to prevent dataset artifacts
    "xgb__min_child_weight": [5, 10, 20],
    "xgb__gamma": [0.1, 0.3, 0.5],
    "xgb__reg_lambda": [2, 5, 10],  # Note: 'lambda' → 'reg_lambda'
    "xgb__reg_alpha": [0, 0.5, 1],  # Note: 'alpha' → 'reg_alpha'
    
    # SAMPLING: Reduce correlation and overfitting
    "xgb__subsample": [0.6, 0.7, 0.8],
    "xgb__colsample_bytree": [0.6, 0.7, 0.8],
    
    # CLASS IMBALANCE: More moderate than 1:5
    "xgb__scale_pos_weight": [1, 2, 3],
    
    # Fixed params - NO prefix needed (or include with prefix)
    # "xgb__tree_method": ["hist"],
    # "xgb__random_state": [42],  # Already set in XGBClassifier above
}

(
    final_model,
    test_metrics,
    val_metrics,
    best_params,
    val_results,
    metadata,
    feature_names
) = grid_search_joblib(
    model=base_pipe,
    param_grid=param_grid,
    train_path=train_data_path_no_mel,
    val_path=test_data_path_no_mel,
    test_path=itw_data_path_no_mel,
    n_jobs=-1,
    verbose=2
)

save_experiment(
    model=final_model,
    metrics=test_metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "final", "XGB_NO_MEL"),
    model_params=best_params,
    feature_names=feature_names,
    metadata_extra=metadata,
    val_results=val_results,
)

print("Metadata:", metadata)
print("Metrics:", test_metrics)

Number of fits: 59049 with n_jobs=-1 parallel jobs


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:  

KeyboardInterrupt: 